In [1]:
%matplotlib inline

In [ ]:
import sys
sys.path

['/workspaces/ennf-fork/toy_mlp/toy_segmentation',
 '/home/codespace/.python/current/lib/python310.zip',
 '/home/codespace/.python/current/lib/python3.10',
 '/home/codespace/.python/current/lib/python3.10/lib-dynload',
 '',
 '/home/codespace/.local/lib/python3.10/site-packages',
 '/home/codespace/.python/current/lib/python3.10/site-packages']

In [9]:
sys.path.append('/workspaces/ennf-fork')

In [10]:
from nn_lib.mdl.loss_functions import CELoss, BCELoss_logits
from nn_lib.optim import Adam, Optimizer
from nn_lib.data import Dataloader

In [11]:
from toy_mlp.toy_segmentation.model_trainer import UNetTrainer
from toy_mlp.mnist_classifier.mnist_mlp_classifier import MnistMLPClassifier
from toy_mlp.toy_segmentation.unet import UNet
from toy_mlp.toy_segmentation.small_unet import SmallUNet
from toy_mlp.toy_segmentation.small_unet_1 import SmallUNet1
from toy_mlp.history_plotter import plot_loss
from toy_mlp.toy_segmentation.PH2_dataset import PH2

from nn_lib.scheduler.multi_step_lr import MultiStepLR

In [12]:
from nn_lib.support_func import iou_score
from nn_lib import Tensor

In [13]:
import numpy as np


In [23]:
a = val_dataset.labels[:25]

In [24]:
a.shape

(25, 16, 16, 1)

In [28]:
a.dtype

dtype('bool')

In [38]:
b1 = a.copy()
b1[:,:,7:] = False

In [39]:
iou_score(Tensor(a,requires_grad=False),Tensor(b1,requires_grad=False)).mean()

0.24

In [44]:
n_epochs=10
optim: Optimizer = Adam
batch_size=25
milestones=[]

In [41]:
# generate a training dataset
size = (16,16)
#size = (32,32)
train_dataset = PH2(ds_type='train',size=size)
# generate a validation dataset different from the training dataset
val_dataset = PH2(ds_type='val',size=size)
# create a dataloader for training data with shuffling and dropping last batch
train_dataloader = Dataloader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
# create a dataloader for validation dataset without shuffling or last batch dropping
val_dataloader = Dataloader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=False)

/workspaces/ennf-fork/toy_mlp/toy_segmentation/PH2Dataset


/workspaces/ennf-fork/toy_mlp/toy_segmentation/PH2Dataset


In [48]:
mlp_model = SmallUNet1(kernum=64)
print(f'Created the following binary MLP classifier:\n{mlp_model}')
# create loss function
loss_fn = BCELoss_logits() #CELoss()
# create optimizer for model parameters
optimizer = optim(mlp_model.parameters(), lr=1e-2, weight_decay=5e-4)
scheduler = MultiStepLR(optimizer, milestones=milestones, gamma=0.5)
# create a model trainer
model_trainer = UNetTrainer(mlp_model, loss_fn, optimizer, scheduler=scheduler)

Created the following binary MLP classifier:
[<nn_lib.mdl.layers.convolution_2d.Conv2d object at 0x7f0487e3fdc0>, <nn_lib.mdl.layers.relu.Relu object at 0x7f046f10d3f0>, <nn_lib.mdl.layers.convolution_2d.Conv2d object at 0x7f046f10f1f0>, <nn_lib.mdl.layers.relu.Relu object at 0x7f046dae9a20>]
[<nn_lib.mdl.layers.convolution_2d.Conv2d object at 0x7f0487e3fc10>, <nn_lib.mdl.layers.relu.Relu object at 0x7f0487e3f9a0>, <nn_lib.mdl.layers.convolution_2d.Conv2d object at 0x7f0487e23280>, <nn_lib.mdl.layers.relu.Relu object at 0x7f0487e22110>]
[<nn_lib.mdl.layers.convolution_2d.Conv2d object at 0x7f0487e23940>, <nn_lib.mdl.layers.relu.Relu object at 0x7f0487e22da0>, <nn_lib.mdl.layers.convolution_2d.Conv2d object at 0x7f0487e23490>, <nn_lib.mdl.layers.relu.Relu object at 0x7f0487e234c0>]
Total number of parameters: 223873


In [49]:
model_trainer.set_datasets(train_dataloader, val_dataloader)
model_trainer.train(1)